# Distance à l'article sur la Philo de Wikipedia

In [30]:
import requests as req
from lxml import html
from urllib.parse import urlparse
from collections import deque

In [31]:
domainUrl = 'https://fr.wikipedia.org'
sourcePath = '/wiki/Peuple'
targetPath = '/wiki/Philosophie'

In [32]:
def getRedirectLinks(htmlText):
    tree = html.fromstring(htmlText)
    links = tree.xpath("//a[@class='mw-redirect']/@href")
    return links

def validateLink(link, domain):
    if len(link) == 0:
        return False, ''

    hrefParsed = urlparse(link)
    if (len(hrefParsed.netloc) > 0 and hrefParsed.netloc != domain) \
            or len(hrefParsed.path) == 0:  # skip anchors in page
        return False, ''
    
    # Only wiki pages
    if not hrefParsed.path.startswith('/wiki'):
        return False, ''
    
    return True, hrefParsed.path

In [33]:
valid, path = validateLink(getRedirectLinks(req.get(domainUrl + sourcePath).content)[0], domainUrl)
valid, path

(True, '/wiki/R%C3%A8gles_de_droit')

## Recherche de distance

In [34]:
def computeDistance(domain, source, target, maxDistance=2):
    visitedPaths = {}  
    searchList = deque()
    searchList.append((source, '')) # No predecessor
    distance = 0
    
    while len(searchList):
        distance += 1
        toSearch = searchList.popleft()
        visitedPaths[toSearch[0]] = toSearch[1] 
        with req.get(domain + toSearch[0]) as response:
            if response.status_code != 200 \
                    or (('content-type' in response.headers) and not response.headers['content-type'].startswith('text/html')):
                print("Erreur de connexion sur : " + toSearch[0] + ", code=" + response.status_code)
                return -2, '', visitedPath
            
            links = getRedirectLinks(response.content)
            valid = False
            for link in links:
                valid, firstLinkPath = validateLink(link, domain)
                if valid:
                    if firstLinkPath.startswith(target):
                        return distance, toSearch[0], visitedPaths
                    elif firstLinkPath not in visitedPaths:
                        if distance < maxDistance:
                            searchList.append((firstLinkPath, toSearch[0]))
                        else:
                            print("Reached max distance for :", toSearch[0])
                            return -3, '', visitedPaths
                        break
            if valid == False:
                print("Pas de nouveau lien valide trouvé sur : " + toSearch[0])
                return -4, '', visitedPaths
    return -1, '', visitedPaths

## Test sur la distance

In [35]:
dist, fromPage, visited = computeDistance(domainUrl, sourcePath, targetPath, 500)

Pas de nouveau lien valide trouvé sur : /wiki/Subdivisions_de_la_R%C3%A9publique_centrafricaine


In [36]:
if dist > 0:
    # Compute visit path
    visitSequence = []
    p = fromPage
    while p != sourcePath:
        visitSequence.append(p)
        p = visited[p][1]

    visitSequence.reverse()
    visitSeqStr = ','.join(iter(visitSequence))
    
    print("La distance à %s depuis %s est de %d, en passant par %s et en visitant %d pages" % (targetPath, sourcePath, dist, visitSeqStr, len(visited)))
else:
    print("Non trouvé ! (%d)" % dist)

Non trouvé ! (-4)


In [37]:
visited, dist, fromPage

({'/wiki/Peuple': '',
  '/wiki/R%C3%A8gles_de_droit': '/wiki/Peuple',
  '/wiki/Droit_coutumier': '/wiki/R%C3%A8gles_de_droit',
  '/wiki/Pr%C3%A9c%C3%A9dent': '/wiki/Droit_coutumier',
  '/wiki/Sources_du_droit': '/wiki/Pr%C3%A9c%C3%A9dent',
  '/wiki/Wikip%C3%A9dia:Guide_d%27internationalisation': '/wiki/Sources_du_droit',
  '/wiki/Wikip%C3%A9dia:GI': '/wiki/Wikip%C3%A9dia:Guide_d%27internationalisation',
  '/wiki/Francophone': '/wiki/Wikip%C3%A9dia:GI',
  '/wiki/Mac%C3%A9doine_(pays)': '/wiki/Francophone',
  '/wiki/Villes_de_la_Mac%C3%A9doine_du_Nord': '/wiki/Mac%C3%A9doine_(pays)',
  '/wiki/Tetovo_(Mac%C3%A9doine)': '/wiki/Villes_de_la_Mac%C3%A9doine_du_Nord',
  '/wiki/R%C3%A9gions_statistiques_de_la_r%C3%A9publique_de_Mac%C3%A9doine_du_Nord': '/wiki/Tetovo_(Mac%C3%A9doine)',
  '/wiki/Municipalit%C3%A9s_de_la_R%C3%A9publique_de_Mac%C3%A9doine': '/wiki/R%C3%A9gions_statistiques_de_la_r%C3%A9publique_de_Mac%C3%A9doine_du_Nord',
  '/wiki/R%C3%A9gion_de_l%27Est_(Mac%C3%A9doine)': '/wiki/Mu